1. Write a python program which searches all the product under a particular product from www.amazon.in. The
product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for
guitars. 

In [120]:
import selenium
import requests
import pandas as pd
from selenium import webdriver
import warnings
warnings.filterwarnings('ignore')
from googlesearch import search
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import re
import os
import selenium.common.exceptions
from selenium.common.exceptions import NoSuchElementException


In [2]:
product = input('Enter the Product Name : ')
driver = webdriver.Chrome()
driver.get('https://www.amazon.in/')

web = driver.find_element(By.XPATH,'/html/body/div[1]/header/div/div[1]/div[2]/div/form/div[2]/div[1]/input')

web.send_keys(product)

search = driver.find_element(By.XPATH,'/html/body/div[1]/header/div/div[1]/div[2]/div/form/div[3]/div/span/input')
search.click()

Enter the Product Name : guitar


2. In the above question, now scrape the following details of each product listed in first 3 pages of your search
results and save it in a data frame and csv. In case if any product has less than 3 pages in search results then
scrape all the products available under that product name. Details to be scraped are: "Brand
Name", "Name of the Product", "Price", "Return/Exchange", "Expected Delivery", "Availability" and
“Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“. 

In [4]:
def product_details(product):
    driver = webdriver.Chrome()
    driver.get('https://www.amazon.in/')

    web = driver.find_element(By.XPATH,'/html/body/div[1]/header/div/div[1]/div[2]/div/form/div[2]/div[1]/input')

    web.send_keys(product)

    search = driver.find_element(By.XPATH,'/html/body/div[1]/header/div/div[1]/div[2]/div/form/div[3]/div/span/input')
    search.click()

    product_data = []

    for page in range(3):
        try:
            products = driver.find_elements(By.XPATH, '//span[@class="a-size-base-plus a-color-base a-text-normal"]')
            prices = driver.find_elements(By.XPATH, '//span[@class="a-price-whole"]')
            availabilities = driver.find_elements(By.XPATH, '//span[@class="a-size-base a-color-price"]')
            delivery = driver.find_elements(By.XPATH, '//span[@class="a-color-base a-text-bold"]')
            urls = driver.find_elements(By.XPATH, '//h2[@class="a-size-mini a-spacing-none a-color-base s-line-clamp-4"]/a')
            return_exc = driver.find_elements(By.XPATH,'//div[@class="a-section a-spacing-none icon-content"]')
            
            for i in range(len(products)):
                product_dict = {
                    "Product Name": products[i].text,
                    "Brand" : products[i].text.split(' ')[0],
                    "Price": prices[i].text if i < len(prices) else '-',
                    "Availability": availabilities[i].text if i < len(availabilities) else '-',
                    "Expected Delivery": delivery[i].text if i < len(delivery) else '-',
                    "Product URL": urls[i].get_attribute('href') if i < len(urls) else '-',
                    "Return/Exchange":return_exc[i].text if i < len(return_exc) else '-'
                }
                product_data.append(product_dict)
        except Exception as e:
            print("An error occurred:", str(e))

        try:
            
            next_button = driver.find_element(By.XPATH, '//a[@class="s-pagination-item s-pagination-next s-pagination-button s-pagination-separator"]')
            next_button.click()
            time.sleep(3)
        except:
            break


    df = pd.DataFrame(product_data)

    df.to_csv(f'{product} on amazon.csv', index=False)
    driver.quit()


product = input('Enter the Product Name : ')
product_details(product)


Enter the Product Name : guitar


3. Write a python program to access the search bar and search button on images.google.com and scrape 10
images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’. 

In [26]:
def save_images(keyword, image_urls):
    folder_path = f"{keyword}_images"
    os.makedirs(folder_path, exist_ok=True)

    for i, url in enumerate(image_urls):
        try:
            response = requests.get(url, timeout=10)
            with open(f"{folder_path}/image{i + 1}.jpg", 'wb') as img_file:
                img_file.write(response.content)
        except Exception as e:
            print(f"Error downloading image: {e}")

def scrape_img(keyword):
    driver = webdriver.Chrome()

    try:
        driver.get('https://www.google.com/imghp')

        driver.implicitly_wait(5)
        
        search_area = driver.find_element(By.NAME, 'q')
        search_area.send_keys(keyword)
        search_area.send_keys(Keys.RETURN)

        driver.implicitly_wait(5)
        
        search_results = search(keyword + " images", num_results=10, lang='en')
        image_urls = [result for result in search_results if result.startswith(('http', 'https'))]

        save_images(keyword, image_urls)

    except Exception as e:
        print(f"Error during scraping: {e}")
        

    finally:
        driver.quit()

keywords = ['fruits', 'cars', 'Machine Learning', 'Guitar', 'Cakes']
for keyword in keywords:
    print(f"Scraping images for {keyword}")
    scrape_img(keyword)
    
print("Image scraping completed.")

Scraping images for fruits
Scraping images for cars
Scraping images for Machine Learning
Scraping images for Guitar
Scraping images for Cakes
Image scraping completed.


4. Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com
and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand
Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”,
“Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the
details is missing then replace it by “- “. Save your results in a dataframe and CSV

In [10]:
def smart_phone(phone):
    driver = webdriver.Chrome()
    try:
        driver.get('https://www.flipkart.com/')

        time.sleep(5)

        search_area = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[1]/div/div/div/div/div[1]/div/div[1]/div/div[1]/header/div[1]/div[2]/form/div/div/input')
        search_area.send_keys(phone)
        search_area.send_keys(Keys.RETURN)

        time.sleep(5)

        product = []

        products = driver.find_elements(By.XPATH, '//div[@class="_4rR01T"]')
        prices = driver.find_elements(By.XPATH, '//div[@class="_30jeq3 _1_WHN1"]')
        ram = driver.find_elements(By.XPATH, '//ul[@class="_1xgFaf"]/li[1]')
        camera = driver.find_elements(By.XPATH, '//ul[@class="_1xgFaf"]/li[3]')
        display = driver.find_elements(By.XPATH, '//ul[@class="_1xgFaf"]/li[2]')
        battery = driver.find_elements(By.XPATH, '//ul[@class="_1xgFaf"]/li[4]')
        urls = driver.find_elements(By.XPATH, '//div[@class="_2kHMtA"]/a')

        for i in range(len(products)):
            try:
                product_dict = {
                    "Brand Name": products[i].text.split(' ')[0],
                    "SmartPhone Name": products[i].text.split('(')[0],
                    "Colour": products[i].text.split('(')[1].split(',')[0],
                    "Price": prices[i].text if prices and i < len(prices) else '-',
                    "RAM": ram[i].text.split('|')[0] if ram and i < len(ram) else '-',
                    "ROM": ram[i].text.split('|')[1] if ram and i < len(ram) else '-',
                    "Primary Camera": camera[i].text.split('|')[0] if camera and i < len(camera) else '-',
                    "Secondary Camera": camera[i].text.split('|')[1] if camera and i < len(camera) else '-',
                    "Display": display[i].text if display and i < len(display) else '-',
                    "Battery": battery[i].text if battery and i < len(battery) else '-',
                    "URL": urls[i].get_attribute('href') if urls and i < len(urls) else '-'
                }

                product.append(product_dict)
            except Exception as e:
                continue

    except Exception as e:
        print(f"Error during scraping: {e}")

    df = pd.DataFrame(product)
    df.to_csv('smartphones_data.csv', index=False)
    print("Data saved to smartphones_data.csv")
    driver.quit()

phone = input("Enter a Smart Phone : ")
smart_phone(phone)


Enter a Smart Phone : Oneplus Nord, pixel 4A
Data saved to smartphones_data.csv


5. Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps. 

In [10]:
def get_coordinates(city):
    driver = webdriver.Chrome()
    driver.get("https://www.google.com/maps")
    
    time.sleep(5)

    search_box = driver.find_element(By.NAME, "q")
    search_box.send_keys(city)
    search_box.send_keys(Keys.RETURN)

    time.sleep(5)

    current_url = driver.current_url
    coordinates = extract_coordinates_from_url(current_url)

    driver.quit()

    return coordinates

def extract_coordinates_from_url(url):
    match = re.search(r'@(-?\d+\.\d+),(-?\d+\.\d+)', url)
    if match:
        latitude = float(match.group(1))
        longitude = float(match.group(2))
        return latitude, longitude
    else:
        return None


city = input("Enter the city name: ")
coordinates = get_coordinates(city)

if coordinates:
    print(f"Coordinates for {city_name}: Latitude {coordinates[0]}, Longitude {coordinates[1]}")
else:
    print(f"Coordinates not found for {city}")

Enter the city name: delhi
Coordinates for delhi: Latitude 28.6442873, Longitude 76.7635522


6. Write a program to scrap all the available details of best gaming laptops from digit.in. 

In [31]:
driver = webdriver.Chrome()
driver.get('https://www.digit.in/')

laptop = driver.find_element(By.XPATH,'/html/body/div[1]/header/div/div[2]/div/nav/ul/li[2]/a')
laptop.click()

best = driver.find_element(By.XPATH,'/html/body/div[1]/header/div/div[2]/div/nav/ul/li[2]/ul/li[3]/a')
best_game = driver.find_element(By.XPATH,'/html/body/div[1]/header/div/div[2]/div/nav/ul/li[2]/ul/li[3]/ul/li[1]/a')
best_game.click()

product =[]

name = driver.find_elements(By.XPATH,'//h3[@class="font130 mt0 mb10 mobilesblockdisplay "]')
operating_system = driver.find_elements(By.XPATH,'//div[@class="woo_code_zone_loop clearbox"]/div[1]/div/span[2]')
display_size = driver.find_elements(By.XPATH,'//div[@class="woo_code_zone_loop clearbox"]/div[2]/div/span[2]')
resolution = driver.find_elements(By.XPATH,'//div[@class="woo_code_zone_loop clearbox"]/div[3]/div/span[2]')
processor = driver.find_elements(By.XPATH,'//div[@class="woo_code_zone_loop clearbox"]/div[4]/div/span[2]')


for i in range(len(name)):
    
    product_dict = {
        "Name" : name[i].text,
        "Operating System" : operating_system[i].text,
        "Display Size" : display_size[i].text,
        "Resolution" : resolution[i].text,
        "Processor" : processor[i].text,
    }
    
    product.append(product_dict)
        
df = pd.DataFrame(product)
df

,Name,Operating System,Display Size,Resolution,Processor
0,Lenovo IdeaPad Gaming 3 82K200X3IN Ryzen 7-580...,Windows 11 Home,15.6,1920 x 1080,5th Gen AMD Ryzen 7-5800
1,Dell New G15-5515 D560805WIN9W Ryzen 5-5600H (...,Windows 10,15.6,1920 x 1080,AMD Ryzen 5-5600H
2,HP Pavilion Gaming 15-ec2145AX Ryzen 5-5600H (...,Windows 11 Home,15.6,1920 x 1080,AMD Ryzen 5 5600H
3,Lenovo IdeaPad Gaming 3 15ACH6 Ryzen 5-5600H (...,Windows 11 Home,15.6,1920 x 1080,AMD Ryzen 5-5600H


In [38]:
driver = webdriver.Chrome()
driver.get('https://www.digit.in/top-products/best-gaming-laptops-40.html')

product =[]

name = driver.find_elements(By.XPATH,'//h3[@class="font130 mt0 mb10 mobilesblockdisplay "]')
operating_system = driver.find_elements(By.XPATH,'//div[@class="woo_code_zone_loop clearbox"]/div/div/span[2]')
display_size = driver.find_elements(By.XPATH,'//div[@class="woo_code_zone_loop clearbox"]/div[2]/div/span[2]')
resolution = driver.find_elements(By.XPATH,'//div[@class="woo_code_zone_loop clearbox"]/div[3]/div/span[2]')
processor = driver.find_elements(By.XPATH,'//div[@class="woo_code_zone_loop clearbox"]/div[4]/div/span[2]')


for i in range(len(name)):
    
    product_dict = {
        "Name" : name[i].text,
        "Operating System" : operating_system[i].text if i < len(operating_system) else '-',
        "Display Size" : display_size[i].text if i < len(display_size) else '-',
        "Resolution" : resolution[i].text if i < len(resolution) else '-',
        "Processor" : processor[i].text if i < len(processor) else '-',
    }
    
    product.append(product_dict)
        
df = pd.DataFrame(product)
df

,Name,Operating System,Display Size,Resolution,Processor
0,HP Omen 17-ck2008TX 13th Gen Core i7-13700HX,Windows 11 Home,17.3,2560 x 1440,13th Gen Intel Core i7-13700HX
1,MSI GT77 Titan 12UHS-054IN 12th Gen Core i9-12...,17.3,16,2560 x 1600,12th Gen Intel Core i7-12700H
2,Lenovo Legion 5i Pro 12th Gen Core i7-12700H (...,2560 x 1440,16,2560 x 1600,13th Gen Intel Core i7-13700HX
3,ASUS ROG Strix Scar 18 G834JZ-N5041WS 13th Gen...,13th Gen Intel Core i7-13700HX,-,-,-
4,Acer Predator Helios Neo 16 13th Gen Core i7-1...,Windows 11 Home,-,-,-
5,ASUS ROG Zephyrus G14 Ryzen 9-6900HS (GA402RJZ...,16,-,-,-
6,MSI Cyborg 15 12th Gen Core i7-12650H (A12VF-2...,2560 x 1600,-,-,-


7. Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be scrapped:
“Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”. 

In [2]:
driver = webdriver.Chrome()
driver.get('https://www.forbes.com/billionaires/')

In [3]:
billionairs =[]
start = 0
end = 14

for i in range(start,end):
    name = driver.find_elements(By.XPATH,'//div[@class="TableRow_rowContainer__IC1Tv"]/div/div[2]/div/div/div[2]')
    rank = driver.find_elements(By.XPATH,'//div[@class="TableRow_rowContainer__IC1Tv"]/div/div[2]/div/div/div[1]')
    net_worth = driver.find_elements(By.XPATH,'//div[@class="TableRow_rowContainer__IC1Tv"]/div/div[2]/div/div/div[3]')
    age = driver.find_elements(By.XPATH,'//div[@class="TableRow_rowContainer__IC1Tv"]/div/div[2]/div/div/div[4]')
    citizenship = driver.find_elements(By.XPATH,'//div[@class="TableRow_rowContainer__IC1Tv"]/div/div[2]/div/div/div[5]')
    source = driver.find_elements(By.XPATH,'//div[@class="TableRow_rowContainer__IC1Tv"]/div/div[2]/div/div/div[6]')
    industry = driver.find_elements(By.XPATH,'//div[@class="TableRow_rowContainer__IC1Tv"]/div/div[2]/div/div/div[7]')
    
    name.pop(1)
    name.pop(1)
    rank.pop(1)
    rank.pop(1)
    rank.pop(1)
    
    for i in range(len(name)):
        
        billionair = {
            "Name" : name[i].text if i < len(net_worth) else '-',
            "Rank" : rank[i].text if i < len(net_worth) else '-',
            "Net Worth" : net_worth[i].text if i < len(net_worth) else '-',
            "Age" : age[i].text if i < len(age) else '-',
            "Citizenship" : citizenship[i].text if i < len(citizenship) else '-',
            "Source" : source[i].text if i < len(source) else '-',
            "Industry" : industry[i].text if i < len(industry) else '-'
        }
        billionairs.append(billionair)
    try:    
        next_button = driver.find_element(By.XPATH,'//span[@class="Pagination_bubbleArrow__WFrX4 Pagination_paginationBtnNext__IOwqm"]') 
        next_button.click()
        
    except Exception:
        break
        time.sleep(3)
        
df = pd.DataFrame(billionairs)
print(df)

                          Name  Rank Net Worth Age    Citizenship  \
0     Bernard Arnault & family     1    $211 B  74         France   
1                    Elon Musk     2    $180 B  51  United States   
2                   Jeff Bezos     3    $114 B  59  United States   
3                Larry Ellison     4    $107 B  78  United States   
4               Warren Buffett     5    $106 B  92  United States   
...                        ...   ...       ...  ..            ...   
2635                   Yu Rong  2540      $1 B  51          China   
2636    Richard Yuengling, Jr.  2540      $1 B  80  United States   
2637             Zhang Gongyun  2540      $1 B  60          China   
2638    Zhang Guiping & family  2540      $1 B  71          China   
2639               Inigo Zobel  2540      $1 B  66    Philippines   

                            Source               Industry  
0                             LVMH       Fashion & Retail  
1                    Tesla, SpaceX             Auto

8. Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted
from any YouTube Video. 

In [41]:
driver = webdriver.Chrome()
driver.get('https://www.youtube.com/watch?v=hlWiI4xVXKY')
time.sleep(5)

for _ in range(30):
    driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
    time.sleep(5)
    
time.sleep(5)

comments = driver.find_elements(By.XPATH, '//yt-formatted-string[@id="content-text"]')
comment_upvotes = driver.find_elements(By.XPATH, '//div[@class="style-scope ytd-comment-action-buttons-renderer"]/span[2]')
comment_times = driver.find_elements(By.XPATH, '//yt-formatted-string[@class="published-time-text style-scope ytd-comment-renderer"]')

comments_data = []

for i in range(len(comments)):
    comment = {
        "Comments": comments[i].text if i < len(comments) else '-',
        "Comment Upvotes": comment_upvotes[i].text if i < len(comment_upvotes) else '-', 
        "Comment Time": comment_times[i].text if i < len(comment_times) else '-'
    }
    comments_data.append(comment)

comments_data = comments_data[:500]

df = pd.DataFrame(comments_data)
print(df)

driver.quit()


                                              Comments Comment Upvotes  \
0    Welcome to Soothing Relaxation! My name is Ped...            1.7K   
1    For this piece I wanted to make a peaceful mor...            1.3K   
2    From the beginning I was immersed in this kind...               3   
3    Oi tudo bem!!\nEu gosto muito dessa música. Ou...                   
4    Absolutely love the soothing vibes of 'Sunny M...               9   
..                                                 ...             ...   
495                     Dios los bendiga por compartir               5   
496  If you are reading this, it doesn't matter whe...               5   
497  I would love waking up to this very nice Thank...                   
498  To all who are reading this, I pray that whate...                   
499  What a beautiful and relaxing music . I love i...              38   

              Comment Time  
0    8 months ago (edited)  
1              4 years ago  
2             3 months a

9. Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in
“London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, overall
reviews, privates from price, dorms from price, facilities and property description. 


In [130]:
driver = webdriver.Chrome()
driver.get('https://www.hostelworld.com/')

In [131]:
page = driver.find_element(By.XPATH,'/html/body/div[3]/div/div/div[2]/div[2]/div[2]/div/div/div/div[1]/div[1]/div/div[2]/input')
page.send_keys('london')

In [133]:
try:
    web = driver.find_element(By.XPATH,'/html/body/div[3]/div/div/div[2]/div[2]/div[2]/div/div/div/div[1]/div[2]/div/ul/li[2]/button/div[2]/div[1]')
    web.click()
except NoSuchElementException as e:
    print("Exception Raised ",e)
    web = driver.find_element(By.CLASS_NAME,"label")
    web.click()

In [134]:
search = driver.find_element(By.XPATH,'/html/body/div[3]/div/div/div[2]/div[2]/div[2]/div/div/div/div[5]/button[1]/div')
search.click()

In [135]:
hostels =[]

try:

    hostel_name = driver.find_elements(By.XPATH,'//div[@class="property-name"]')
    distance = driver.find_elements(By.XPATH,'//span[@class="distance-description"]')
    ratings = driver.find_elements(By.XPATH,'//span[@class="number"]')
    total_reviews = driver.find_elements(By.XPATH,'//span[@class="left-margin"]')
    overall_reviews = driver.find_elements(By.XPATH,'//span[@class="keyword"]')
    privates_price = driver.find_elements(By.XPATH,'//div[@class="property-accommodation-prices"]/div[1]')
    dorms_price = driver.find_elements(By.XPATH,'//div[@class="property-accommodation-prices"]/div[2]')
    
    for i in range(len(hostel_name)):
        
        hostel = {
            "Hostel Name" : hostel_name[i].text if i < len(hostel_name) else '-',
            "Distace from City Center" : distance[i].text if i < len(distance) else '-',
            "Rating" : ratings[i].text if i < len(ratings) else '-',
            "Total Reviews" : total_reviews[i].text if i < len(total_reviews) else '-',
            "Overall Reviews" : overall_reviews[i].text if i < len(overall_reviews) else '-',
            "Private from Price" : privates_price[i].text if i < len(privates_price) else '-',
            "Dorms from Price" : dorms_price[i].text if i < len(dorms_price) else '-'
        }
        hostels.append(hostel)

                
except Exception as e:
    print("Error ",str(e))

finally:
    driver.quit()
    
df = pd.DataFrame(hostels)
df.to_csv('Hostel_data.csv',index=False)
print("Scrapping complete")

Scrapping complete
